# SQL

## Conexión a PostgreSQL usando SQLAlchemy

In [4]:
import os
import pandas as pd
from typing import Union
from sqlalchemy import create_engine, text

# Reemplaza USER y PASSWORD del .env
engine = create_engine("sqlite:///data/sql-murder-mystery.db")

# Probar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar: {e}")


def ejecutar_query(query: str) -> Union[pd.DataFrame, str]:
    """
    Ejecuta una consulta SQL en la base de datos y devuelve un DataFrame.

    Parámetros:
    - query (str): La consulta SQL a ejecutar.

    Retorna:
    - Union[pd.DataFrame, pl.DataFrame, str]: Retorna un DataFrame en pandas o polars dependiendo del parámetro 'devolver_polars',
      o un mensaje de éxito/error si la operación no es un SELECT.
    """
    
    with engine.connect() as connection:
        try:
            result = connection.execute(text(query))
            
            # Determinar si es una consulta SELECT para devolver datos
            if result.returns_rows:
                rows = result.fetchall()
                columns = result.keys()

                # Convertir filas en un diccionario de listas para crear correctamente el DataFrame en polars
                data_dict = {col: [row[idx] for row in rows] for idx, col in enumerate(columns)}

                # Verificación de coincidencia de columnas y filas
                return pd.DataFrame(data_dict)
            else:
                connection.commit()
                return f"Query ejecutada correctamente: {result.rowcount} filas afectadas."
        except Exception as e:
            return f"Error al ejecutar la query: {e}"


Conexión exitosa


In [6]:
query = 'SELECT * FROM crime_scene_report WHERE date = 20180115 AND city = "SQL City"'

ejecutar_query(query=query)

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [3]:
query =SELECT * FROM person WHERE address_street_name = "Northwestern Dr"  ORDER BY address_number DESC LIMIT 1


ejecutar_query(query=query)

SyntaxError: invalid syntax (3037353974.py, line 1)

In [ ]:
query = ""

ejecutar_query(query=query)

In [ ]:
query= ''

ejecutar_query(query=query)

In [ ]:
query= ''

ejecutar_query(query=query)

In [ ]:
query= ''

ejecutar_query(query=query)

In [ ]:
query= ''

ejecutar_query(query=query)

In [ ]:
query= ""

ejecutar_query(query=query)